In [ ]:
%pip install -qqq keras tensorflow numpy matplotlib pandas seaborn scikit-learn plotly nbformat

In [ ]:
!wget https://archive.ics.uci.edu/static/public/53/iris.zip -O ../data/iris/iris.zip

In [ ]:
!unzip ../data/iris/iris.zip -d ../data/iris

In [ ]:
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="white", color_codes=True)

column_names = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm', 'Species']
iris = pd.read_csv("../data/iris/iris.data", names=column_names)

iris.head()

In [ ]:
iris["Species"].value_counts()

In [ ]:
iris.plot(kind="scatter", x="SepalLengthCm", y="SepalWidthCm")

In [ ]:
sns.jointplot(x="SepalLengthCm", y="SepalWidthCm", data=iris, size=5)

In [ ]:
sns.FacetGrid(iris, hue="Species", height=5) \
   .map(plt.scatter, "SepalLengthCm", "SepalWidthCm") \
   .add_legend()

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=0)
X_2d = tsne.fit_transform(iris.iloc[:, :-1])

iris_2d = pd.DataFrame(X_2d, columns=["x", "y"])
iris_2d["Species"] = iris["Species"]

sns.FacetGrid(iris_2d, hue="Species", height=5) \
   .map(plt.scatter, "x", "y") \
   .add_legend()

In [ ]:
import plotly.graph_objects as go

tsne = TSNE(n_components=3, random_state=0)
X_3d = tsne.fit_transform(iris.iloc[:, :-1])

iris_3d = pd.DataFrame(X_3d, columns=["x", "y", "z"])
iris_3d["Species"] = iris["Species"]
iris_3d.head()


In [ ]:
# Convert categorical labels to numerical values
iris_3d['label_numeric'] = pd.factorize(iris_3d['Species'])[0]

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Scatter3d(
    x=iris_3d['x'],
    y=iris_3d['y'],
    z=iris_3d['z'],
    mode='markers',
    marker=dict(
        color=iris_3d['label_numeric'],
        size=2,
        colorscale='Viridis',
        opacity=0.8
    )
)])

fig.show()

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

import sqlite3

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler

import tensorflow as tf
import keras

In [ ]:
Y = iris['Species']
X = iris.drop(['Species'], axis=1)
print("Shape of Input  features: {}".format(X.shape))
print("Shape of Output features: {}".format(Y.shape))

In [ ]:
label_results = []
Y_encoded = []

for y in Y:
    if y == 'Iris-setosa':
        label_results.append([1, 0, 0])
        Y_encoded.append(0)
    elif y == 'Iris-versicolor':
        label_results.append([0, 1, 0])
        Y_encoded.append(1)
    else:
        label_results.append([0, 0, 1])
        Y_encoded.append(2)

Y_final = np.array(label_results)
Y_encoded = np.array(Y_encoded)


print("Therefore, our final shape of output feature will be {}".format(Y_final.shape))

In [ ]:
Y_encoded

In [ ]:
seed = 42
np.random.seed(seed)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y_final, test_size=0.25, random_state=seed, stratify=Y_encoded, shuffle=True)

print("Training Input shape\t: {}".format(x_train.shape))
print("Testing Input shape\t: {}".format(x_test.shape))
print("Training Output shape\t: {}".format(y_train.shape))
print("Testing Output shape\t: {}".format(y_test.shape))

In [ ]:
std_clf = StandardScaler()
# x_train_new = std_clf.fit_transform(x_train)
# x_test_new = std_clf.transform(x_test)

x_train_new = x_train
x_test_new = x_test

In [ ]:
model = keras.models.Sequential()
model.add(keras.Input(shape=(4,)))
model.add(keras.layers.Dense(10, activation=keras.activations.relu, kernel_initializer='he_normal', 
                                kernel_regularizer=keras.regularizers.l2(0.01)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(7, activation=keras.activations.relu, kernel_initializer='he_normal', 
                                kernel_regularizer=keras.regularizers.l1_l2(l1=0.001, l2=0.001)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(5, activation=keras.activations.relu, kernel_initializer='he_normal', 
                                kernel_regularizer=keras.regularizers.l1_l2(l1=0.001, l2=0.001)))
model.add(keras.layers.Dense(3, activation=keras.activations.softmax))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

iris_model = model.fit(x_train_new, y_train, epochs=700, batch_size=7)

In [ ]:
model.evaluate(x_test_new, y_test)

In [ ]:
model.summary()

In [ ]:
model.save("../data/iris/iris_model.keras")

In [ ]:
model2 = keras.models.load_model("../data/iris/iris_model.keras")
model2.summary()

In [ ]:
x_test_new

In [ ]:
result = model2(np.array([[4.4, 3.2, 1.3, 0.2]]))

In [ ]:
max_index = np.argmax(result)
print("Predicted Class: {}".format(lbl_clf.inverse_transform([max_index])[0]))

In [ ]:
iris.iloc[42]

In [ ]:
classes = {i: lbl_clf.classes_[i] for i in range(len(lbl_clf.classes_))}
classes